<img src="../Imagenes/1.jpg">

# Proyecto Pandas - Ataque de tiburones

## Origen y descripción de los datos

El .csv a importar aporta datos sobre ataques de tiburones recopilados por <i>Global Shark Attack File</i>. 

Para más información sobre la base de datos consultar <a href="http://www.sharkattackfile.net/">aquí</a>.

## Preparación de los datos

<img src="../Imagenes/p1.jpg">

### Librerías

Importamos las librerías necesarias para el desarrollo de la tarea.

In [1]:
import pandas as pd
import numpy as np
import re
import ipywidgets as widgets
from IPython.display import display
from ipywidgets import interact, interact_manual

### Exploración y primera toma de contacto con los datos

* Importamos el dataset y lo renombramos.

In [2]:
shark_attacks = pd.read_csv("../Datos/GSAF5.csv", engine = "python") 

* Este comando nos mostrará el tamaño total de nuestra base de datos.

In [3]:
shark_attacks.shape

(5992, 24)

* Vemos el nombre de las columnas de nuestro <i>dataset</i>.

In [4]:
shark_attacks.columns

Index(['Case Number', 'Date', 'Year', 'Type', 'Country', 'Area', 'Location',
       'Activity', 'Name', 'Sex ', 'Age', 'Injury', 'Fatal (Y/N)', 'Time',
       'Species ', 'Investigator or Source', 'pdf', 'href formula', 'href',
       'Case Number.1', 'Case Number.2', 'original order', 'Unnamed: 22',
       'Unnamed: 23'],
      dtype='object')

Podemos empezar a observar que los nombres de las columnas no están normalizados. Posibles cosas a arreglar serían las minúsculas/mayúsculas, borrar los espacios que se ven al final y estandarizar la puntuación por '_'.

* Vemos los tipos de datos que hay en las diferentes columnas.

In [5]:
shark_attacks.dtypes

Case Number               object
Date                      object
Year                       int64
Type                      object
Country                   object
Area                      object
Location                  object
Activity                  object
Name                      object
Sex                       object
Age                       object
Injury                    object
Fatal (Y/N)               object
Time                      object
Species                   object
Investigator or Source    object
pdf                       object
href formula              object
href                      object
Case Number.1             object
Case Number.2             object
original order             int64
Unnamed: 22               object
Unnamed: 23               object
dtype: object

* Hacemos una primera visualización de los datos

In [6]:
shark_attacks.head()

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
0,2016.09.18.c,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,male,M,...,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.c-NSB.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.18.c,2016.09.18.c,5993,NaN,NaN
1,2016.09.18.b,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,Chucky Luciano,M,...,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.b-Luciano.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.18.b,2016.09.18.b,5992,NaN,NaN
2,2016.09.18.a,18-Sep-16,2016,Unprovoked,USA,Florida,"New Smyrna Beach, Volusia County",Surfing,male,M,...,NaN,"Orlando Sentinel, 9/19/2016",2016.09.18.a-NSB.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.18.a,2016.09.18.a,5991,NaN,NaN
3,2016.09.17,17-Sep-16,2016,Unprovoked,AUSTRALIA,Victoria,Thirteenth Beach,Surfing,Rory Angiolella,M,...,NaN,"The Age, 9/18/2016",2016.09.17-Angiolella.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.17,2016.09.17,5990,NaN,NaN
4,2016.09.15,16-Sep-16,2016,Unprovoked,AUSTRALIA,Victoria,Bells Beach,Surfing,male,M,...,2 m shark,"The Age, 9/16/2016",2016.09.16-BellsBeach.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.09.16,2016.09.15,5989,NaN,NaN


In [7]:
shark_attacks.tail()

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
5987,ND.0005,Before 1903,0,Unprovoked,AUSTRALIA,Western Australia,Roebuck Bay,Diving,male,M,...,NaN,"H. Taunton; N. Bartlett, p. 234",ND-0005-RoebuckBay.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0005,ND.0005,6,NaN,NaN
5988,ND.0004,Before 1903,0,Unprovoked,AUSTRALIA,Western Australia,NaN,Pearl diving,Ahmun,M,...,NaN,"H. Taunton; N. Bartlett, pp. 233-234",ND-0004-Ahmun.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0004,ND.0004,5,NaN,NaN
5989,ND.0003,1900-1905,0,Unprovoked,USA,North Carolina,Ocracoke Inlet,Swimming,Coast Guard personnel,M,...,NaN,"F. Schwartz, p.23; C. Creswell, GSAF",ND-0003-Ocracoke_1900-1905.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0003,ND.0003,4,NaN,NaN
5990,ND.0002,1883-1889,0,Unprovoked,PANAMA,NaN,"Panama Bay 8�N, 79�W",NaN,Jules Patterson,M,...,NaN,"The Sun, 10/20/1938",ND-0002-JulesPatterson.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0002,ND.0002,3,NaN,NaN
5991,ND.0001,1845-1853,0,Unprovoked,CEYLON (SRI LANKA),Eastern Province,"Below the English fort, Trincomalee",Swimming,male,M,...,NaN,S.W. Baker,ND-0001-Ceylon.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,ND.0001,ND.0001,2,NaN,NaN


Comparando los primeros registros con los últimos y podemos observar como <CODE>'Case Number'</CODE> y <CODE>Date</CODE> presentan formatos diferentes, <CODE>'Year'</CODE> tiene valores a 0, en <CODE>Name</CODE> hay elementos que deberían ir en <CODE>Sex</CODE>, etc. Pasaremos a observar los atributos que nos interesen con más detalle más adelante.

Las columnas <CODE>'name'</CODE>, <CODE>'fatal_(y/n)'</CODE>, <CODE>'time'</CODE>, <CODE>'investigator_or_source'</CODE>, <CODE>'pdf'</CODE>, <CODE>'href_formula'</CODE>, <CODE>'href'</CODE>,<CODE>'case_number_1'</CODE>,<CODE>'case_number_2'</CODE>, <CODE>'original_order'</CODE> no aportan información relevante para analizar.

* Con esto vemos la cantidad de <CODE>'NaN'</CODE> con los que nos encontramos.

In [8]:
shark_attacks.isnull().sum()

Case Number                  0
Date                         0
Year                         0
Type                         0
Country                     43
Area                       402
Location                   496
Activity                   527
Name                       200
Sex                        567
Age                       2681
Injury                      27
Fatal (Y/N)                 19
Time                      3213
Species                   2934
Investigator or Source      15
pdf                          0
href formula                 1
href                         3
Case Number.1                0
Case Number.2                0
original order               0
Unnamed: 22               5991
Unnamed: 23               5990
dtype: int64

Primeras observaciones en relación a los <CODE>NaN</CODE>:


* Vemos que las columnas <CODE>'unnamed_22'</CODE> y <CODE>'unnamed_23'</CODE> tienen casi todos los valores <CODE>NaN</CODE>.


* La columna <CODE>'Time'</CODE> tiene más de la mitad de los registros como <CODE>NaN</CODE>. 


* Las columnas <CODE>'Age'</CODE> y <CODE>'Species'</CODE> se acercan a la mitad de registros como <CODE>NaN</CODE>.


## Limpieza de datos

La <b>limpieza de datos </b> es el proceso que sufren los datos en los que se eliminan los que sean erróneos, redundantes o incompletos.

### Normalización de los nombres de los atributos

In [9]:
def clean (words):
    
    """
    ****************************************************
    Función para limpiar la información,
    pasar la primera a mayúculas y el resto a minúscula,
    eliminaremos espacios innecesarios, el resto serán
    sustituidos por '_', igual que otros elementos de
    puntuación?
    
    Input : lista
    Output : lista
    ****************************************************
    
    """
    c = []
    for word in words:
        word = str(word).lower()
        word = word.strip()
        word = word.replace(".", ' ')
        word = word.replace(" ", '_')
        word = word.replace(":", '_')
        word = word.replace("__", '_')
        word = word.replace ("/", "")
        word = word.replace ("?", "")
        word = word.replace ("(", "")
        word = word.replace (")", "")
        
        c.append (word)
    return c

In [10]:
def change_names (list1, list2):
    
    """
    *****************************************
    Convierte a diccionario dos listas dadas
    *****************************************
    """
    
    dictchange = dict(zip(list(list1), list(list2)))
    return dictchange

In [11]:
#Renombramos las columnas.
shark_attacks = shark_attacks.rename(columns = change_names(shark_attacks.columns, clean(shark_attacks.columns)))

In [12]:
shark_attacks.columns

Index(['case_number', 'date', 'year', 'type', 'country', 'area', 'location',
       'activity', 'name', 'sex', 'age', 'injury', 'fatal_yn', 'time',
       'species', 'investigator_or_source', 'pdf', 'href_formula', 'href',
       'case_number_1', 'case_number_2', 'original_order', 'unnamed_22',
       'unnamed_23'],
      dtype='object')

### Normalización y transformación de datos 

En este apartado nos centraremos en normalizar algunas de las columnas que pueden interesarnos. 

#### Year

Vemos como los datos referentes a la fecha del ataque del tiburon no presentan el mismo formato, intentaremos extraer los años de forma estandarizada para facilitar su posterior análisis.

In [13]:
shark_attacks['year'].value_counts()

2015    139
2011    128
2014    125
0       124
2013    122
       ... 
1742      1
1758      1
1818      1
1822      1
1595      1
Name: year, Length: 232, dtype: int64

In [14]:
"""
**************************************************
En los casos en los que el campo 'year' está a 0, 
buscamos si en 'date' está el año.
Si se encuentra con dos fechas hace una media.
Si no encuentra información en ninguna de las columnas
añadirá la media.
**************************************************
"""

year_lst = []
year_lst1 = list(shark_attacks['year'])
        
for row in shark_attacks['date']:
    temp_row = ''.join(re.findall('\-*\d{4}',row))
    temp_row = re.sub('\-','',temp_row)

    if len(str(temp_row))>4:
        temp_row = int((int(temp_row[0:4]) + int(temp_row [4:8]))/2)
        year_lst.append(temp_row)
    else:
        year_lst.append(temp_row)

normalized_years = []
e = 0       
for year in year_lst1:
    try:
        if year != 0:
            normalized_years.append(int(year))
            e+=1
        elif year == 0:
            normalized_years.append(int(year_lst[e]))
            e+=1
    except ValueError:
        normalized_years.append((int(np.mean(normalized_years))))
        
        
shark_attacks['year'] = normalized_years

#### Type

El atributo <CODE>'type' </CODE> está bastante limpio, solo cambiaremos <CODE>'Boating'</CODE> por <CODE>'Boat</CODE>.

In [15]:
shark_attacks['type'].value_counts()

Unprovoked      4386
Provoked         557
Invalid          519
Sea Disaster     220
Boat             200
Boating          110
Name: type, dtype: int64

In [16]:
shark_attacks['type_attack'] = [re.sub("Boat", "Boating",re.sub("ing", "", i)) for i in shark_attacks['type']]

In [17]:
shark_attacks['type_attack'].value_counts()

Unprovoked      4386
Provoked         557
Invalid          519
Boating          310
Sea Disaster     220
Name: type_attack, dtype: int64

#### Atributos del accidentado

##### Sex

Analizamos los valores que nos encontramos en <CODE>'sex'</CODE> para proceder a su normalización. Veremos cuantos valores hay de cada una de las opciones. También tendremos en cuenta los <b>567</b> valores <CODE>'NaN'</CODE> que vimos durante el proceso de exploración.

In [18]:
shark_attacks['sex'].value_counts()

M      4835
F       585
M         2
lli       1
N         1
.         1
Name: sex, dtype: int64

Ejecutar solo una de las dos opciones:

###### Opción 1:

Para normalizar el resto de elementos, modificaremos las observaciones en función del porcentaje de cada sexo.

In [19]:
sex_list=[]
for sex in shark_attacks['sex']:
    
    """
    ******************************************************************
    Con este primer paso inclumos el mayor número de valores correctos
    eliminando los posibles espacios que encontremos.
    ******************************************************************
    """
    
    sex = str(sex)
    if sex.strip() == "M" or sex.strip() == "F":
        sex_list.append(sex.strip())

En esta opción nos centraremos en:

1- sacar % de <CODE>'M'</CODE> y de <CODE>'F'</CODE>,                                         
2- buscar la forma de repartir ese % en el resto de observaciones.

In [20]:
# % de 'm' y de 'f'
c=round(sex_list.count("F")/len(sex_list)*int(len(shark_attacks['sex'])-len(sex_list)))
d=round(sex_list.count("M")/len(sex_list)*int(len(shark_attacks['sex'])-len(sex_list)))

# creamos una nueva lista con los elementos faltantes en proporción a los % que hemos obtenido antes.
list1 = ("F "* c).split()
list2 = ("M "* d).split()
listrest = list1 + list2

#añadimos la lista a los valores que ya tenemos para poder cambiar luego los valores.
normalized_sex = []
e=0
for sex in shark_attacks['sex']:
    sex= str(sex)
    sex = str(sex.strip())
    if sex == "M" or sex == "F":
        normalized_sex.append(sex)
    else:
        normalized_sex.append(listrest[e])
        e+=1

In [21]:
temp_normalized_sex = []
for sex in normalized_sex:
    #Cambiamos los valores a :
    if sex == "M":
        sex = "Male"
        temp_normalized_sex.append(sex)
    elif sex == "F":
        sex = "Female"
        temp_normalized_sex.append(sex)
normalized_sex = temp_normalized_sex

In [22]:
#Cambiamos los valores de la columna
shark_attacks['sex'] = normalized_sex

In [23]:
shark_attacks['sex'].value_counts()

Male      5346
Female     646
Name: sex, dtype: int64

###### Opción 2:

En esta opción sustituiremos los casos desconocidos por <CODE>'unknown'</CODE>

In [ ]:
normalized_sex = []
for sex in shark_attacks['sex']:
    sex = str(sex)
    if sex.strip() == "M" or sex.strip() == "F":
        normalized_sex.append(sex.strip())
    else:
        normalized_sex.append('Unknown')

In [ ]:
temp_normalized_sex = []
for sex in normalized_sex:
    #Cambiamos los valores a :
    if sex == "M":
        sex = "Male"
        temp_normalized_sex.append(sex)
    elif sex == "F":
        sex = "Female"
        temp_normalized_sex.append(sex)
    else:
        temp_normalized_sex.append(sex)
normalized_sex = temp_normalized_sex

In [ ]:
#Cambiamos los valores de la columna
shark_attacks['sex'] = normalized_sex

Comprobamos que no tenemos <CODE>'NaN'</CODE>

In [ ]:
shark_attacks['sex'].value_counts()

##### Age

Para normalizar <CODE>'age'</CODE> analizaremos primero el tipo de valores que nos encontramos y la frecuencia con la que aparecen.

In [24]:
pd.unique(shark_attacks['age'])

array(['16', '36', '43', nan, '60s', '51', '50', '12', '9', '22', '25',
       '37', '20', '49', '15', '21', '40', '72', '18', '29', '31', '11',
       '10', '59', '42', '34', '35', '19', '6', '27', '64', '60', '23',
       '52', '13', '57', '48', '39', '24', '26', '69', '46', 'Teen', '41',
       '45', '65', '38', '71', '32', '58', '28', '54', '44', '14', '7',
       '62', '40s', '68', '47', '17', '30', '63', '70', '18 months', '53',
       '20s', '33', '30s', '50s', '8', '61', '55', 'teen', '66', '77',
       '74', '3', '56', '28 & 26', '5', '86', '18 or 20', '12 or 13',
       '46 & 34', '28, 23 & 30', 'Teens', '36 & 26', '8 or 10', '84',
       '� ', ' ', '30 or 36', '6�', '21 & ?', '75', '33 or 37', 'mid-30s',
       '73', '23 & 20', '7      &    31', '20?', "60's", '32 & 30',
       '16 to 18', '87', '67', 'Elderly', 'mid-20s', 'Ca. 33', '21 or 26',
       '>50', '18 to 22', 'adult', '9 & 12', '? & 19', '9 months',
       '25 to 35', '23 & 26', '1', '(adult)', '33 & 37', '25 or 2

In [25]:
shark_attacks['age'].value_counts()

17          148
18          145
19          138
20          136
15          135
           ... 
60's          1
30 or 36      1
23 & 26       1
2�            1
33 & 26       1
Name: age, Length: 151, dtype: int64

In [26]:
shark_attacks['age']= clean(shark_attacks['age'])
pd.unique(shark_attacks['age'])

array(['16', '36', '43', 'nan', '60s', '51', '50', '12', '9', '22', '25',
       '37', '20', '49', '15', '21', '40', '72', '18', '29', '31', '11',
       '10', '59', '42', '34', '35', '19', '6', '27', '64', '60', '23',
       '52', '13', '57', '48', '39', '24', '26', '69', '46', 'teen', '41',
       '45', '65', '38', '71', '32', '58', '28', '54', '44', '14', '7',
       '62', '40s', '68', '47', '17', '30', '63', '70', '18_months', '53',
       '20s', '33', '30s', '50s', '8', '61', '55', '66', '77', '74', '3',
       '56', '28_&_26', '5', '86', '18_or_20', '12_or_13', '46_&_34',
       '28,_23_&_30', 'teens', '36_&_26', '8_or_10', '84', '�', '',
       '30_or_36', '6�', '21_&_', '75', '33_or_37', 'mid-30s', '73',
       '23_&_20', '7___&__31', "60's", '32_&_30', '16_to_18', '87', '67',
       'elderly', 'mid-20s', 'ca_33', '21_or_26', '>50', '18_to_22',
       'adult', '9_&_12', '_&_19', '9_months', '25_to_35', '23_&_26', '1',
       '33_&_37', '25_or_28', '37,_67,_35,_27,__&_27', '21,_

Vemos que los datos han sido tratados de forma muy diferente. Intentaremos agruparlos por categorías en el apartado de discretización.

##### Country

Pasaremos el filtro de limpieza para normalizar los nombres de los paises.

In [27]:
pd.unique(shark_attacks['country'])

array(['USA', 'AUSTRALIA', 'NEW CALEDONIA', 'REUNION', 'BAHAMAS', 'SPAIN',
       'CHINA', 'JAPAN', 'COLUMBIA', 'SOUTH AFRICA', 'EGYPT',
       'NEW ZEALAND', 'INDONESIA', 'FRENCH POLYNESIA', 'CAPE VERDE',
       'Fiji', 'BRAZIL', 'DOMINICAN REPUBLIC', 'CAYMAN ISLANDS',
       'UNITED ARAB EMIRATES', 'ARUBA', 'MOZAMBIQUE', 'THAILAND', 'FIJI',
       'PUERTO RICO', 'ITALY', 'MEXICO', 'ATLANTIC OCEAN', 'GREECE',
       'MAURITIUS', nan, 'ST. MARTIN', 'FRANCE', 'ECUADOR',
       'PAPUA NEW GUINEA', 'TRINIDAD & TOBAGO', 'KIRIBATI', 'ISRAEL',
       'DIEGO GARCIA', 'TAIWAN', 'JAMAICA', 'PALESTINIAN TERRITORIES',
       'GUAM', 'SEYCHELLES', 'BELIZE', 'PHILIPPINES', 'NIGERIA', 'TONGA',
       'SCOTLAND', 'CANADA', 'CROATIA', 'SAUDI ARABIA', 'CHILE',
       'ANTIGUA', 'KENYA', 'RUSSIA', 'TURKS & CAICOS', 'COSTA RICA',
       'UNITED KINGDOM', 'MALAYSIA', 'UNITED ARAB EMIRATES (UAE)',
       'SAMOA', 'AZORES', 'SOLOMON ISLANDS', 'SOUTH KOREA', 'MALTA',
       'VIETNAM', 'MADAGASCAR', 'PANAMA',

In [28]:
shark_attacks['country'].value_counts()

USA                 2116
AUSTRALIA           1279
SOUTH AFRICA         565
PAPUA NEW GUINEA     133
NEW ZEALAND          125
                    ... 
INDIAN OCEAN?          1
BRITISH ISLES          1
EGYPT / ISRAEL         1
GREENLAND              1
WESTERN SAMOA          1
Name: country, Length: 203, dtype: int64

In [29]:
#Limpiaremos los nombres
shark_attacks['country'] = [(str(i).title()).replace("_"," ")
                                           for i in clean(shark_attacks['country'])]

Nos pasa algo parecido que con las edades, los datos han sido tratados de forma diferente, no todos los valores son países. Podría ser interesante hacer una discretización por continentes.

#### Otros atributos

In [30]:
def nan_to_unknown (column):
    
    """
    ******************************
    Convierte los NaN en 'unknown'
    ******************************
    """
    
    w = column.fillna('Unknown')
    return w

In [31]:
shark_attacks['species'] = nan_to_unknown(shark_attacks['species'])

In [32]:
shark_attacks['area'] = nan_to_unknown(shark_attacks['area'])

In [33]:
shark_attacks['location'] = nan_to_unknown(shark_attacks['location'])

In [34]:
shark_attacks['activity'] = nan_to_unknown(shark_attacks['activity'])

### Discretización 

La <b>discretización</b> nos permite establecer un creterio por el cual podemos dividir los valores de un atributo en dos o más conjuntos.

#### Ages

En este punto intentaremos agrupar la edad de las víctimas por rangos:
* <CODE>'Newborn'</CODE>: agrupará valores de 0 a 1 año,
* <CODE>'Children'</CODE>: agrupará valores de 1 a 10 años,
* <CODE>'Teen'</CODE>: agrupará valores de 10 a 19 años
* <CODE>'Young'</CODE>: agrupará valores de los 20 a los 30 años,
* <CODE>'Adult'</CODE>: agrupará valores de los 31 a los 55 años,
* <CODE>'Senior'</CODE>: agrupará valores de los 56 a los 75 años,
* <CODE>'Old'</CODE>: agrupará valores desde los 75 años.

In [35]:
age_list = []
ages_list = ['newborn', 'children','teen','young','adult','senior','old']
        
for row in shark_attacks['age']: 
    
    get_row = "".join(re.findall('(months)',str(row)))
    
    if row.lower() in ages_list:
        age_list.append(row.capitalize()) 

    else:
        
        age_row = "".join(re.findall('\d{1,2}',str(row)))
        age_row = re.sub('\?','',age_row)
        age_row = re.sub('\&','',age_row)
        
        try:
            if get_row == "months":
                age_list.append('Newborn')
            elif int(age_row) >= 1 and int(age_row) <= 10:
                age_list.append('Children')
            elif int(age_row)>10 and int(age_row)<20:
                age_list.append('Teen')
            elif int(age_row)>=20 and int(age_row)<=30:
                age_list.append('Young')
            elif int(age_row)>30 and int(age_row)<=55:
                age_list.append('Adult')
            elif int(age_row)>55 and int(age_row)<75:
                age_list.append('Senior')
            elif int(age_row)>=75:
                age_list.append('Old')

        except ValueError:
            age_list.append('Unknown')

In [36]:
shark_attacks['age'] = age_list

#### Year to Century

Crearemos una columna nueva para agrupar los años por siglos. Crearemos una columna nueva con esta información a la que llamaremos <CODE>'century</CODE>.

Esto nos será útil para su posterior visualización.

In [48]:
centuries = []
for year in shark_attacks['year']:
    
    if year <= 100:
        centuries.append('I')
    elif year <= 200 and year > 100:
        centuries.append('II')
    elif year <= 300 and year > 200:
        centuries.append('III')
    elif year <= 400 and year > 300:
        centuries.append('IV')
    elif year <= 500 and year > 400:
        centuries.append('V')
    elif year <= 600 and year > 500:
        centuries.append('VI')
    elif year <= 700 and year > 600:
        centuries.append('VII')
    elif year <= 800 and year > 700:
        centuries.append('VIII')
    elif year <= 900 and year > 800:
        centuries.append('IX')
    elif year <= 1000 and year > 900:
        centuries.append('X')
    elif year <= 1100 and year > 1000:
        centuries.append('XI')
    elif year <= 1200 and year > 1100:
        centuries.append('XII')
    elif year <= 1300 and year > 1200:
        centuries.append('XIII')
    elif year <= 1400 and year > 1300:
        centuries.append('XIV')
    elif year <= 1500 and year > 1400:
        centuries.append('XV')
    elif year <= 1600 and year > 1500:
        centuries.append('XVI')
    elif year <= 1700 and year > 1600:
        centuries.append('XVII')
    elif year <= 1800 and year > 1700:
        centuries.append('XVIII')
    elif year <= 1900 and year > 1800:
        centuries.append('XIX')
    elif year <= 2000 and year > 1900:
        centuries.append('XX')
    elif year <= 2100 and year > 2000:
        centuries.append('XXI')
    
shark_attacks['century'] = centuries

### Reducción de la dimensionalidad

En este último paso antes de empezar con el análisis se busca trabajar con menos datos y obtener los mismos resultados. Para ello pasaremos a eliminar atributos que no nos aportan información relevante.


Descartamos las columnas <CODE>'date'</CODE>, <CODE>'fatal_(y/n)'</CODE>, <CODE>'time'</CODE>, <CODE>'investigator_or_source'</CODE>, <CODE>'pdf'</CODE>, <CODE>'href_formula'</CODE>, <CODE>'href'</CODE>,<CODE>'case_number_1'</CODE>,<CODE>'case_number_2'</CODE>, <CODE>'original_order'</CODE>, <CODE>'unnamed_22'</CODE> y <CODE>'unnamed_23'</CODE>.

La columna <CODE>'name'</CODE> también será eliminada por la RGPD.

Y pasaremos a ordenar las columnas.

In [38]:
shark_attacks = shark_attacks.drop(['date','name','type', 'injury', 'fatal_yn', 'time',
                                    'investigator_or_source', 'pdf', 'href_formula', 'href',
                                    'case_number_1', 'case_number_2', 'original_order', 'unnamed_22',
                                    'unnamed_23'], axis=1)

In [39]:
#Reordenamos las columnas:

column_order = ['case_number','year','century','sex','age',
                'country','area','location', 'activity',
                'type_attack', 'species']

shark_attacks = shark_attacks[column_order]

In [40]:
shark_attacks.isnull().sum()

case_number    0
year           0
century        0
sex            0
age            0
country        0
area           0
location       0
activity       0
type_attack    0
species        0
dtype: int64

Buscaremos las observaciones duplicadas y las eliminaremos.

In [41]:
shark_attacks = shark_attacks.drop_duplicates()
shark_attacks.shape

(5990, 11)

Guardamos nuestros datos en un CSV.

In [42]:
shark_attacks.to_csv("../Datos/shark_attacks.csv", index = False)

## [BONUS] Visualización de resultados

Como hemos visto un poco por encima los witgets intentaremos incluirlos para visualizar los datos.

In [43]:
ALL = 'All'
def unique_values_plus_ALL(array):
    
    """
    ********************************
    Esta función saca los valores
    únicos de una columna y le añade
    All.
    
    Input = array
    Output = valores únicos del array
    + All.
    ********************************
    """
    
    unique = array.unique().tolist()
    unique.insert(0, ALL)
    return unique

Crearemos un primer desplegable que saque las estadísticas por columna.

In [44]:
@interact
# El @interact es necesario para que funcione.
def describe(column=list(shark_attacks.columns)):
    
    """
    ********************************************
    Con esta función creamos un desplegable, al 
    seleccionar una de las columnas nos devolverá
    los datos agregados
    *********************************************
    """
    
    print(shark_attacks[column].describe())

interactive(children=(Dropdown(description='column', options=('case_number', 'year', 'century', 'sex', 'age', …

Con las siguientes líneas de código conseguimos que nos devuelva los datos según el filtro que le indiquemos.

In [45]:
lista_century = unique_values_plus_ALL(shark_attacks.century)
lista_age = unique_values_plus_ALL(shark_attacks.age)
lista_sex = unique_values_plus_ALL(shark_attacks.sex)
lista_type_attack = unique_values_plus_ALL(shark_attacks.type_attack)

In [46]:
#funcion
def filtro(century, age, sex, type_attack):
    
    """
    **************************
    Aplica el filtro a las 3
    columnas que nos interesan.
    **************************
    
    """
    
    if (century == ALL) & (age == ALL) & (sex == ALL)& (type_attack == ALL):
        filtrado = shark_attacks
        
    elif (century == ALL):
        filtrado = shark_attacks.loc[(shark_attacks['age'] == age)& 
                             (shark_attacks['sex'] == sex) & (shark_attacks['type_attack'] == type_attack)]
    elif (age == ALL):
        filtrado = shark_attacks.loc[(shark_attacks['century'] == age)& 
                             (shark_attacks['sex'] == sex)&(shark_attacks['type_attack'] == type_attack)]
    elif (sex == ALL):
        filtrado = shark_attacks.loc[(shark_attacks['century'] == age)& 
                             (shark_attacks['age'] == sex)&(shark_attacks['type_attack'] == type_attack)]
    elif (type_attack == ALL):
        filtrado = shark_attacks.loc[(shark_attacks['century'] == age)& 
                             (shark_attacks['age'] == sex)&(shark_attacks['sex'] == sex)]
    else:
        filtrado = shark_attacks.loc[(shark_attacks['century'] == century) &
                             (shark_attacks['age'] == age) & 
                             (shark_attacks['sex'] == sex) & (shark_attacks['type_attack'] == type_attack)]
        
    return filtrado

Filtro final:

In [47]:
interact (filtro, 
          century = lista_century,
          age = lista_age,
          sex = lista_sex, 
          type_attack = lista_type_attack);

interactive(children=(Dropdown(description='century', options=('All', 'XXI', 'XX', 'XIX', 'XVIII', 'XVII', 'XV…